# Classifier Model Test

## Model Weight Check

In [ ]:
import sys
import os
import glob
import time
import torch
from torchvision import models
import pandas as pd
from thop import profile
import tempfile
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import copy  # deepcopy를 사용하기 위해 추가

sys.path.append('../')
from model.loader import model_Loader  # 필요에 따라 수정

ckpt_dir = '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint'

model_dict = {
    'resnet' : models.resnet34(weights = models.ResNet34_Weights.IMAGENET1K_V1),
    'mobilenet' : models.mobilenet_v3_large(weights = models.MobileNet_V3_Large_Weights.IMAGENET1K_V2),
    'efficient' : models.efficientnet_v2_l(weights = models.EfficientNet_V2_L_Weights.IMAGENET1K_V1),
    'convnext' : models.convnext_large(weights = models.ConvNeXt_Large_Weights.IMAGENET1K_V1), 
    'swin-transformer' : models.swin_v2_b(weights = models.Swin_V2_B_Weights.IMAGENET1K_V1),  
    'vision-transformer' : models.vit_l_16(weights = models.ViT_L_16_Weights.IMAGENET1K_V1),
    'maxvit' : models.maxvit_t(weights = models.MaxVit_T_Weights.IMAGENET1K_V1)
}

# 초기 딕셔너리 설정
params_dict = {model_name: 0 for model_name in model_dict.keys()}
inference_dict = params_dict.copy()
inference_time_dict = params_dict.copy()
flops_dict = {}
memory_dict = {}
model_size_dict = {}
throughput_dict = {}
layer_count_dict = {}
accuracy_dict = {}

def params_count(model):
    return sum(p.numel() for p in model.parameters())

def inference_count(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def measure_inference_time(model, input_tensor, device='cpu', num_runs=100):
    model = model.to(device)
    model.eval()
    input_tensor = input_tensor.to(device)
    
    with torch.no_grad():
        # 워밍업
        for _ in range(10):
            _ = model(input_tensor)
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        start_time = time.time()
        
        for _ in range(num_runs):
            _ = model(input_tensor)
        
        if device.type == 'cuda':
            torch.cuda.synchronize()
        end_time = time.time()
    
    total_time = end_time - start_time
    avg_time = total_time / num_runs
    return avg_time

def calculate_flops(model, input_tensor):
    # 모델의 복사본을 생성하고 CPU로 이동
    model_cpu = copy.deepcopy(model).cpu()
    # 입력 텐서를 CPU로 이동
    input_cpu = input_tensor.cpu()
    model_cpu.eval()
    with torch.no_grad():
        flops, params = profile(model_cpu, inputs=(input_cpu,))
    return flops

def measure_memory_usage(model, input_tensor, device='cpu'):
    model = model.to(device)
    model.eval()
    input_tensor = input_tensor.to(device)
    
    torch.cuda.empty_cache()
    if device.type == 'cuda':
        torch.cuda.reset_peak_memory_stats(device)
    
    with torch.no_grad():
        _ = model(input_tensor)
    
    if device.type == 'cuda':
        mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)  # MB 단위
    else:
        mem = 0  # CPU 메모리 사용량 측정은 별도로 필요
    
    return mem

def get_model_size(model):
    with tempfile.NamedTemporaryFile() as tmp:
        torch.save(model.state_dict(), tmp.name)
        tmp.seek(0, os.SEEK_END)
        size = tmp.tell() / (1024 ** 2)  # MB 단위
    return size

def count_layers(model):
    return len(list(model.modules())) - 1  # 최상위 모델 제외

def calculate_throughput(avg_inference_time, batch_size=1):
    return batch_size / avg_inference_time

# 입력 데이터 준비
input_size = (1, 3, 224, 224)
input_tensor = torch.randn(input_size)

# 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 정확도 측정을 위한 데이터셋 준비 (예: CIFAR-10)
transform = transforms.Compose([
    transforms.Resize(224),
    # transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# 모든 모델에 대해 지표 측정
for model_name, model in model_dict.items():
    print(f"Processing {model_name}...")
    # 파라미터 수 측정
    params_dict[model_name] = params_count(model)
    
    # 인퍼런스 가능 파라미터 수 측정
    inference_dict[model_name] = inference_count(model)
    
    # 인퍼런스 시간 측정
    avg_inference_time = measure_inference_time(model, input_tensor, device=device, num_runs=100)
    inference_time_dict[model_name] = avg_inference_time
    print(f"{model_name}: Average Inference Time = {avg_inference_time:.6f} seconds")
    
    # FLOPs 측정
    try:
        flops = calculate_flops(model, input_tensor)
        flops_dict[model_name] = flops
        print(f"{model_name}: FLOPs = {flops}")
    except Exception as e:
        flops_dict[model_name] = None
        print(f"{model_name}: FLOPs 측정 중 오류 발생 - {e}")
    
    # 메모리 사용량 측정
    mem = measure_memory_usage(model, input_tensor, device=device)
    memory_dict[model_name] = mem
    print(f"{model_name}: Memory Usage = {mem} MB")
    
    # 모델 크기 측정
    size = get_model_size(model)
    model_size_dict[model_name] = size
    print(f"{model_name}: Model Size = {size:.2f} MB")
    
    # Throughput 측정
    throughput = calculate_throughput(avg_inference_time, batch_size=1)
    throughput_dict[model_name] = throughput
    print(f"{model_name}: Throughput = {throughput:.2f} samples/sec")
    
    # 레이어 수 측정
    layers = count_layers(model)
    layer_count_dict[model_name] = layers
    print(f"{model_name}: Number of Layers = {layers}")
    
    print("-" * 50)  # 구분선 추가

# DataFrame으로 변환
params_df = pd.DataFrame(params_dict.items(), columns=['model', 'Parameters'])
inference_df = pd.DataFrame(inference_dict.items(), columns=['model', 'Trainable Parameters'])
inference_time_df = pd.DataFrame(inference_time_dict.items(), columns=['model', 'Avg Inference Time (s)'])
flops_df = pd.DataFrame(flops_dict.items(), columns=['model', 'FLOPs'])
memory_df = pd.DataFrame(memory_dict.items(), columns=['model', 'Memory Usage (MB)'])
model_size_df = pd.DataFrame(model_size_dict.items(), columns=['model', 'Model Size (MB)'])
throughput_df = pd.DataFrame(throughput_dict.items(), columns=['model', 'Throughput (samples/sec)'])
layers_df = pd.DataFrame(layer_count_dict.items(), columns=['model', 'Number of Layers'])

# 모든 데이터프레임 합치기
result_df = params_df.merge(inference_df, on='model') \
                     .merge(inference_time_df, on='model') \
                     .merge(flops_df, on='model') \
                     .merge(memory_df, on='model') \
                     .merge(model_size_df, on='model') \
                     .merge(throughput_df, on='model') \
                     .merge(layers_df, on='model')

# 인덱스를 모델 이름으로 설정
result_df.set_index('model', inplace=True)

print(result_df)


In [ ]:
result_df.to_csv('/mnt/hdd/octc/PCOS_experiment/multi_v3/metrics/model_params.csv')

In [ ]:
import subprocess
import tempfile
import os
import shutil

# 제공된 LaTeX 코드 (소수점 3자리로 수정됨)
latex_code = r"""
\documentclass{standalone}
\usepackage{booktabs}
\usepackage{siunitx}

\sisetup{
    round-mode          = places,
    round-precision     = 3,
}

\begin{document}
\begin{table}[htbp]
    \centering
    \caption{Model Metrics Comparison}
    \label{tab:model_metrics}
    \begin{tabular}{l
                    S[table-format=8.0]
                    S[table-format=8.0]
                    S[table-format=1.3]
                    S[table-format=10.0]
                    S[table-format=4.3]
                    S[table-format=5.2]
                    S[table-format=8.3]
                    S[table-format=3.0]}
        \toprule
        \textbf{Model} & \textbf{Parameters} & \textbf{Trainable Parameters} & \textbf{Avg Inference Time (s)} & \textbf{FLOPs} & \textbf{Memory Usage (MB)} & \textbf{Model Size (MB)} & \textbf{Throughput (samples/sec)} & \textbf{Number of Layers} \\
        \midrule
        resnet & 21797672 & 21797672 & 0.003 & 3678739456 & 103.312 & 83.286 & 364.821 & 115 \\
        mobilenet & 5483032 & 5483032 & 0.003 & 234838456 & 244.871 & 21.113 & 331.254 & 254 \\
        efficient & 118515272 & 118515272 & 0.017 & 12379301696 & 798.776 & 454.599 & 58.461 & 1408 \\
        convnext & 197767336 & 197767336 & 0.004 & 34389508608 & 1458.106 & 754.544 & 220.788 & 382 \\
        swin-transformer & 87930848 & 87930848 & 0.014 & 10270570496 & 1780.294 & 336.377 & 69.733 & 431 \\
        vision-transformer & 304326632 & 304326632 & 0.006 & 39856041984 & 2929.969 & 1161.028 & 159.113 & 295 \\
        maxvit & 30919624 & 30919624 & 0.012 & 5456315648 & 3176.244 & 118.776 & 83.416 & 669 \\
        \bottomrule
    \end{tabular}
\end{table}
\end{document}
"""

# 출력 경로 설정
output_dir = "/mnt/hdd/octc/PCOS_experiment/multi_v3/metrics/"
output_svg_path = os.path.join(output_dir, "model_metrics.svg")

# 임시 디렉토리 생성 (출력 디렉토리와 분리)
with tempfile.TemporaryDirectory() as tmpdirname:
    print(f"임시 디렉토리 생성: {tmpdirname}")
    
    # LaTeX 파일 작성
    tex_file = os.path.join(tmpdirname, "model_metrics.tex")
    with open(tex_file, "w") as f:
        f.write(latex_code)
    print("LaTeX 파일 작성 완료.")
    
    # LaTeX 컴파일 (PDF 생성)
    try:
        print("LaTeX 파일을 PDF로 컴파일 중...")
        subprocess.run(
            ["pdflatex", "-interaction=nonstopmode", "model_metrics.tex"],
            cwd=tmpdirname,
            check=True,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        print("PDF 컴파일 완료.")
    except subprocess.CalledProcessError as e:
        print("LaTeX 컴파일 중 오류 발생:")
        print(e.stdout.decode())
        print(e.stderr.decode())
        exit(1)
    
    # PDF 파일 경로 확인
    pdf_file = os.path.join(tmpdirname, "model_metrics.pdf")
    if not os.path.exists(pdf_file):
        print("PDF 파일이 생성되지 않았습니다.")
        exit(1)
    
    # SVG 변환 (pdf2svg 또는 Inkscape 사용)
    svg_file = os.path.join(tmpdirname, "model_metrics.svg")
    try:
        # pdf2svg가 설치되어 있는지 확인
        if shutil.which("pdf2svg"):
            print("pdf2svg를 사용하여 PDF를 SVG로 변환 중...")
            subprocess.run(
                ["pdf2svg", "model_metrics.pdf", "model_metrics.svg"],
                cwd=tmpdirname,
                check=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            print("SVG 변환 완료 (pdf2svg 사용).")
        elif shutil.which("inkscape"):
            # pdf2svg가 없으면 Inkscape 사용
            print("pdf2svg가 설치되어 있지 않습니다. Inkscape를 사용하여 변환 중...")
            subprocess.run(
                ["inkscape", "model_metrics.pdf", "--export-type=svg", "--export-filename=model_metrics.svg"],
                cwd=tmpdirname,
                check=True,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
            print("SVG 변환 완료 (Inkscape 사용).")
        else:
            raise FileNotFoundError("pdf2svg와 Inkscape가 모두 설치되어 있지 않습니다.")
    except subprocess.CalledProcessError as e:
        print("PDF를 SVG로 변환하는 중 오류 발생:")
        print(e.stdout.decode())
        print(e.stderr.decode())
        exit(1)
    except FileNotFoundError as e:
        print(e)
        exit(1)
    
    # SVG 파일 경로 확인
    if not os.path.exists(svg_file):
        print("SVG 파일이 생성되지 않았습니다.")
        exit(1)
    
    # 출력 디렉토리 생성 (없으면)
    os.makedirs(output_dir, exist_ok=True)
    
    # SVG 파일 이동
    shutil.move(svg_file, output_svg_path)
    print(f"SVG 파일이 성공적으로 저장되었습니다: {output_svg_path}")


# Metric 평가

In [ ]:
#!/usr/bin/env python
import os
import sys
# Add parent directory to sys.path for module imports
sys.path.append('../')
import warnings
import torch
from lib.seed import set_seed
from lib.evaluation.eval import dataloader_builder, evaluation

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torchvision")
warnings.filterwarnings("ignore", category=FutureWarning)

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEED = 42
ROOT_DIR = '/mnt/hdd/octc/PCOS_Dataset'
SAVE_METRIC_DIR = '/mnt/hdd/octc/PCOS_experiment/multi_v3/metrics'
FOLD_NUM = 5
OUTER_LAYER_NUM = 3

# Initialize
set_seed(SEED)

# Define evaluation configurations
evaluation_configs = [
    # {
    #     "description": "origin",
    #     "csv_path": os.path.join(ROOT_DIR, 'test.csv'),
    #     "mask_use": False,
    #     "model_suffix": "origin",
    #     "checkpoint_root_dir" : '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint',
    #     "data_type" : "Dataset"
    
    # },
    # {
    #     "description": "mask",
    #     "csv_path": os.path.join(ROOT_DIR, 'test.csv'),
    #     "mask_use": True,
    #     "model_suffix": "mask",
    #     "checkpoint_root_dir" : '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint',
    #     "data_type" : "Dataset"
    # },
    # {
    #     "description": "maskwithaugment",
    #     "csv_path": os.path.join(ROOT_DIR, 'test.csv'),
    #     "mask_use": True,
    #     "model_suffix": "maskwithaugment",
    #     "checkpoint_root_dir" : '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint',
    #     "data_type" : "Dataset"
    # },
    {
        "description": "originwithaugment",
        "csv_path": os.path.join(ROOT_DIR, 'test.csv'),
        "mask_use": False,
        "model_suffix": "originwithaugment",
        "checkpoint_root_dir" : '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint',
        "data_type" : "Dataset"
    },
    # {
    #     "description": "inpaint",
    #     "csv_path": os.path.join(ROOT_DIR, 'test.csv'),
    #     "mask_use": False,
    #     "model_suffix": "inpaint",
    #     "checkpoint_root_dir" : '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint',
    #     "data_type" : "Dataset_OCI"
    # },
    # {
    #     "description": "inpaintwithaugment",
    #     "csv_path": os.path.join(ROOT_DIR, 'test.csv'),
    #     "mask_use": False,
    #     "model_suffix": "inpaintwithaugment",
    #     "checkpoint_root_dir" : '/mnt/hdd/octc/PCOS_experiment/multi_v3/checkpoint',
    #     "data_type" : "Dataset_OCI"
    # },
]

def run_evaluation(config):
    test_loader = dataloader_builder(
        test_csv_path=config["csv_path"],
        root_dir=ROOT_DIR,
        input_res=(3, 224, 224),
        bs_size=32,
        mask_use=config["mask_use"],
        class_num=OUTER_LAYER_NUM,
        data_type = config["data_type"]
        
    )
    filtered_model_cards = [
        card for card in sorted(os.listdir(config["checkpoint_root_dir"]))
        if card.split('-')[-1] == config["model_suffix"]
    ]
    # for i in range(len(filtered_model_cards)):
    #     print(filtered_model_cards[i])
    num_models = len(filtered_model_cards) // FOLD_NUM
    print(f"Number of models: {num_models}")
    evaluation(
        outlayer_num=OUTER_LAYER_NUM,
        model_cards=filtered_model_cards,
        checkpoint_root_dir=config["checkpoint_root_dir"],
        mask_use=config["mask_use"],
        device=DEVICE,
        test_loader=test_loader,
        save_metric_dir=SAVE_METRIC_DIR
    )
    

# Execute evaluations
for config in evaluation_configs:
    run_evaluation(config)


# 전체 메트릭 비교 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import numpy as np

def get_significance_stars(p_value):
    """Return significance stars based on p-value."""
    if p_value < 1e-4:
        return '****'
    elif p_value < 1e-3:
        return '***'
    elif p_value < 1e-2:
        return '**'
    elif p_value < 0.05:
        return '*'
    else:
        return 'ns'  # not significant

def visualize_box_plot_with_p_value(df):
    """
    Visualize box plots for each metric grouped by version with annotated p-values.

    Parameters:
    - df (pd.DataFrame): DataFrame containing metrics and a 'Version' column.
    """
    import itertools  # Moved inside the function for self-containment

    # Required metrics
    metrics = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']

    # Ensure all required columns exist
    for metric in metrics + ['Version']:
        if metric not in df.columns:
            raise ValueError(f"Column '{metric}' not found in the DataFrame.")

    # Convert metric columns to numeric, coercing errors to NaN
    df[metrics] = df[metrics].apply(pd.to_numeric, errors='coerce')

    # Drop rows with NaN in any of the metric columns or 'Version'
    df = df.dropna(subset=metrics + ['Version'])

    # Get unique versions in reverse order
    versions = sorted(df['Version'].unique(), reverse=True)
    num_versions = len(versions)

    if num_versions < 2:
        raise ValueError("At least two unique versions are required for comparison.")

    # Define subplot grid based on number of metrics
    cols = 3
    rows = int(np.ceil(len(metrics) / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
    axes = axes.flatten()  # Flatten in case of multiple rows

    for idx, metric in enumerate(metrics):
        ax = axes[idx]

        # Prepare data for boxplot
        data = [df[df['Version'] == version][metric].dropna().values for version in versions]

        # Create box plot
        bp = ax.boxplot(data, patch_artist=True, labels=versions)

        # Customize boxplot appearance
        # Define colors for the base version and other versions
        base_color = 'skyblue'
        other_colors = ['lightgreen', 'salmon', 'gold', 'plum', 'lightgrey']
        colors = [base_color] + other_colors[:num_versions - 1]

        for patch, color in zip(bp['boxes'], colors):
            patch.set_facecolor(color)

        ax.set_title(f'{metric}')
        ax.set_xlabel('Version')
        ax.set_ylabel(metric)

        # Perform t-tests between the first version and others
        base_data = data[0]
        y_max = max([d.max() for d in data])  # Current maximum y-value for the metric

        # Calculate the range for y-axis
        y_min = min([d.min() for d in data])
        y_range = y_max - y_min
        y_offset_initial = y_range * 0.05  # 5% of the range for initial offset
        y_offset_increment = y_range * 0.05  # 5% increment for each additional annotation

        # Initialize the starting y-position for annotations
        current_y = y_max + y_offset_initial

        for i in range(1, num_versions):
            compare_data = data[i]
            t_stat, p_value = ttest_ind(base_data, compare_data, equal_var=False)  # Welch's t-test

            significance = get_significance_stars(p_value)

            # Coordinates for annotation
            x1, x2 = 1, i + 1  # Base version is at position 1
            h = y_range * 0.01  # Height of the annotation line
            col = 'k'  # Color of the annotation lines and text

            # Draw the annotation lines
            ax.plot([x1, x1, x2, x2], [current_y, current_y + h, current_y + h, current_y], lw=1.3, c=col)

            # Annotate the p-value and significance
            # ax.text((x1 + x2) * 0.5, current_y + h, f'p = {p_value:.2e}\n{significance}', ha='center', va='bottom', fontsize=10)
            ax.text((x1 + x2) * 0.5, current_y + h, f'{significance}', ha='center', va='bottom', fontsize=10)

            # Increment the y-position for the next annotation
            current_y += y_offset_increment

        # Adjust y-axis limit to accommodate all annotations
        ax.set_ylim(top=current_y + y_offset_increment)

    # Remove any unused subplots
    for idx in range(len(metrics), len(axes)):
        fig.delaxes(axes[idx])

    plt.tight_layout()
    plt.show()


## Binary & Visualize

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import pandas as pd
import pickle

# binary_metric_save 함수는 이전에 수정된 버전을 사용합니다.

def binary_metric_save(save_metric_dir: str, pickles: list):
    # Initialize DataFrame to store results
    all_metrics = pd.DataFrame(columns=['Model', 'Version', 'Fold', 'Accuracy', 'Precision',
                                        'Sensitivity', 'Specificity', 'F1-score', 'AUC'])
    roc_metrics = {}
    # Read data from pickle files and add to DataFrame
    for idx, pk in enumerate(pickles):
        pickle_path = os.path.join(save_metric_dir, pk)
        # Use regex to extract model_name, version, fold_num
        match = re.match(r'(.+?)_(.+?)_(.+?)\.pkl$', pk)
        
        if match:
            model_name, version, fold_num = match.groups()
        else:
            print(f"Filename {pk} does not match the expected pattern. Skipping.")
            continue  # Skip files that don't match the expected pattern

        # Load data from pickle file
        with open(pickle_path, 'rb') as f:
            metric = pickle.load(f)

        roc_metrics[f'{model_name}_{version}_{fold_num}'] = {
            'fpr': metric['FPR'],
            'tpr': metric['TPR'],
            'roc_auc': metric['AUC'],
            'thresholds': metric['Thresholds']
        }
        # Convert metric data to DataFrame
        metric_df = pd.DataFrame(metric)

        # Select required columns and the last row
        metric_df = metric_df[['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']].iloc[-1]

        # Add additional info: Model, Version, Fold
        metric_df['Model'] = model_name
        metric_df['Version'] = version
        metric_df['Fold'] = pd.to_numeric(fold_num, errors='coerce')  # Ensure 'Fold' is numeric

        # Reorder columns
        metric_df = metric_df[['Model', 'Version', 'Fold', 'Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']]

        # Append to all_metrics DataFrame
        all_metrics = pd.concat([all_metrics, metric_df.to_frame().T], ignore_index=True)

    # Exclude 'Fold' from mean calculation
    avg_metrics = all_metrics.drop(columns=['Fold']).groupby(['Model', 'Version'], as_index=False).mean()

    # Sort by AUC in descending order
    avg_metrics = avg_metrics.sort_values(by='AUC', ascending=False)

    # Round numeric columns to 3 decimal places
    numeric_cols = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']
    avg_metrics[numeric_cols] = avg_metrics[numeric_cols].apply(pd.to_numeric, errors='coerce').round(3)

    return roc_metrics, all_metrics, avg_metrics

# 저장된 metric 파일이 있는 디렉토리 경로
save_metric_dir = '/mnt/hdd/octc/PCOS_experiment/binary_v2/metrics/'
# 디렉토리 내 모든 pickle 파일 목록을 정렬
pickles = sorted(os.listdir(save_metric_dir))

# metrics 데이터를 불러오는 함수
roc_metrics, all_metrics, avg_metrics = binary_metric_save(save_metric_dir, pickles)
avg_metrics.to_csv('/mnt/hdd/octc/PCOS_experiment/binary_v2/all_metrics.csv', index=False)

# 예제 사용
visualize_box_plot_with_p_value(all_metrics)

## Multi & Visualize

In [ ]:
import os
import re
import pickle
import pandas as pd
from typing import List, Tuple


def multi_class_metric_save(save_metric_dir: str, pickles: List[str]) -> Tuple[dict, pd.DataFrame, pd.DataFrame]:
    # Initialize DataFrame to store results
    all_metrics = pd.DataFrame(columns=['Model', 'Version', 'Fold', 'Accuracy', 'Precision',
                                        'Sensitivity', 'Specificity', 'F1-score', 'AUC'])
    roc_metrics = {}
    
    # Read data from pickle files and add to DataFrame
    for idx, pk in enumerate(pickles):
        pickle_path = os.path.join(save_metric_dir, pk)
        
        # Use regex to extract model_name, version, fold_num
        match = re.match(r'(.+?)_(.+?)_(.+?)\.pkl$', pk)
        
        if match:
            model_name, version, fold_num = match.groups()
            if version == 'transformer':
                version = 'mask-ft-medsam1'
        else:
            print(f"Filename {pk} does not match the expected pattern. Skipping.")
            continue  # Skip files that don't match the expected pattern

        # Load data from pickle file
        with open(pickle_path, 'rb') as f:
            metric = pickle.load(f)
        
        # Add ROC metrics if available
        if 'FPR' in metric and 'TPR' in metric and 'AUC' in metric and 'Thresholds' in metric:
            roc_metrics[f'{model_name}_{version}_{fold_num}'] = {
                'fpr': metric['FPR'],
                'tpr': metric['TPR'],
                'roc_auc': metric['AUC'],
                'thresholds': metric['Thresholds']
            }
        
        # Convert metric data to DataFrame and check available keys
        available_metrics = {key: metric[key] for key in ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC'] if key in metric}
        metric_df = pd.DataFrame([available_metrics])

        # Add additional info: Model, Version, Fold
        metric_df['Model'] = model_name
        metric_df['Version'] = version
        metric_df['Fold'] = pd.to_numeric(fold_num, errors='coerce')  # Ensure 'Fold' is numeric

        # Reorder columns
        metric_df = metric_df[['Model', 'Version', 'Fold'] + [col for col in ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC'] if col in metric_df]]

        # Append to all_metrics DataFrame
        all_metrics = pd.concat([all_metrics, metric_df], ignore_index=True)

    # Exclude 'Fold' from mean calculation
    avg_metrics = all_metrics.drop(columns=['Fold']).groupby(['Model', 'Version'], as_index=False).mean()

    # Sort by AUC in descending order if AUC is available
    if 'AUC' in avg_metrics.columns:
        avg_metrics = avg_metrics.sort_values(by='AUC', ascending=False)

    # Round numeric columns to 3 decimal places
    # numeric_cols = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score', 'AUC']
    numeric_cols = ['Accuracy', 'Precision', 'Sensitivity', 'Specificity', 'F1-score']
    for col in numeric_cols:
        if col in avg_metrics.columns:
            avg_metrics[col] = avg_metrics[col].round(3)

    return roc_metrics, all_metrics, avg_metrics

type = "augment"
# 저장된 metric 파일이 있는 디렉토리 경로
save_metric_dir = f'/mnt/hdd/octc/PCOS_experiment/multi_v3/metrics/{type}'

# 디렉토리 내 모든 pickle 파일 목록을 정렬
pickles = sorted(os.listdir(save_metric_dir))
# metrics 데이터를 불러오는 함수
roc_metrics, all_metrics, avg_metrics = multi_class_metric_save(save_metric_dir, pickles)

# AUC 소수점 3자리까지
avg_metrics.sort_values(by='Sensitivity', ascending=False, inplace=True)
all_metrics.sort_values(by='Sensitivity', ascending=False, inplace=True)
avg_metrics = avg_metrics.round(3)
all_metrics = all_metrics.round(3)
# # # avg_metrics['AUC'] = avg_metrics['AUC'].round(3)
all_metrics.to_csv(f'/mnt/hdd/octc/PCOS_experiment/multi_v3/all_{type}_metrics.csv', index=False)
avg_metrics.to_csv(f'/mnt/hdd/octc/PCOS_experiment/multi_v3/avg_{type}_metrics.csv', index=False)
visualize_box_plot_with_p_value(df = all_metrics)

In [56]:
from glob import glob 

metrics = glob('/mnt/hdd/octc/PCOS_experiment/multi_v3/metrics/all_*.csv')

# 모든 csv 파일을 읽어서 DataFrame으로 변환
all_metrics = pd.concat([pd.read_csv(metric) for metric in metrics], ignore_index=True)
all_metrics.sort_values(by='Sensitivity', ascending=False, inplace=True, ignore_index=True)
all_metrics.to_csv('/mnt/hdd/octc/PCOS_experiment/multi_v3/metrics/all_overall_metrics.csv')
# convenxt만 뽑기
convnext_metrics = all_metrics[all_metrics['Model'] == 'convnext']

convnext_metrics

,Model,Version,Fold,Accuracy,Precision,Sensitivity,Specificity,F1-score,AUC
4,convnext,origin,4,75.000,73.299,75.000,79.188,71.860,79.146
5,convnext,inpaintwithaugment,4,74.689,72.368,74.689,78.677,71.649,76.969
7,convnext,inpaint,4,74.689,72.368,74.689,78.677,71.649,76.969
8,convnext,inpaint,3,74.534,73.262,74.534,77.679,71.075,72.002
11,convnext,inpaintwithaugment,3,74.534,73.262,74.534,77.679,71.075,72.002
15,convnext,origin,3,74.224,71.385,74.224,77.899,70.269,76.311
32,convnext,origin,2,73.758,72.957,73.758,76.797,69.669,76.936
38,convnext,inpaint,5,73.602,70.174,73.602,78.575,70.458,75.501
39,convnext,originwithaugment,1,73.602,70.853,73.602,80.007,71.456,75.923
41,convnext,inpaintwithaugment,5,73.602,70.174,73.602,78.575,70.458,75.501


#  ETC 

In [ ]:
import pandas as pd 
pd.read_csv('/mnt/hdd/octc/experiment/binary.csv')

In [ ]:
import sys, os 
sys.path.append('../')
import torchvision.transforms as transforms
from lib.datasets.ds_tools import kfold_extract
from lib.dataset import Custom_stratified_Dataset
from torchsampler import ImbalancedDatasetSampler


from lib.datasets.sampler import class_weight_sampler, class_weight_getter
from torch.utils.data import DataLoader
folds, train_df, test_df  = kfold_extract('/mnt/hdd/octc/experiment/binary.csv',  5, True, random_state =627)
for idx, fold in enumerate(folds):
    print(f'Fold {idx+1}')
    # fold안에 레이블 분포 확인
    print(fold['train']['label'].value_counts())
    print(fold['val']['label'].value_counts())
    print('-------------------')
    dataset = Custom_stratified_Dataset(
        df = fold['train'],
        root_dir = '/mnt/hdd/octc/BACKUP/Dataset',
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor()
        ])
    )
    
    # sampler = class_weight_sampler(fold)
    
    train_loader = DataLoader(
        dataset,
        batch_size = 10,
        sampler = ImbalancedDatasetSampler(dataset),
        num_workers = 8
    )
    
    for img, label in train_loader:
        print(label.unique(return_counts=True))
    print('-------------------')
    # print(class_weight_getter(fold))
    
        
    
    
    print('-------------------')
    dataset
    
    
    
    break 

In [ ]:
import sys, os 
sys.path.append('../')
import torchvision.transforms as transforms
import numpy as np
import torch
device = 'cuda'
from lib.datasets.ds_tools import kfold_extract
from lib.dataset import Custom_stratified_Dataset
from lib.datasets.sampler import class_weight_sampler, class_weight_getter
from torch.utils.data import DataLoader
folds, train_df, test_df  = kfold_extract('/mnt/hdd/octc/experiment/binary.csv',  5, True, random_state =627)
for idx, fold in enumerate(folds):
    print(f'Fold {idx+1}')
    # fold안에 레이블 분포 확인
    print(fold['train']['label'].value_counts())
    print(fold['val']['label'].value_counts())
    print('-------------------')
    dataset = Custom_stratified_Dataset(
        df = fold['train'],
        root_dir = '/mnt/hdd/octc/BACKUP/Dataset',
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor()
        ])
    )
    
    n_pos = (fold['train']['label']==1).sum()
    n_neg = (fold['train']['label']==0).sum()
    pos_weight_value = n_neg / n_pos
    pos_weight = torch.tensor([pos_weight_value], dtype=torch.float32).to('cuda')

    
    print(pos_weight)
    print('-------------------')
    
    
    
    break 

In [ ]:
from sklearn.model_selection import GroupKFold, KFold
import pandas as pd
def k_fold_split(csv_path, random_seed =42 ):
    df = pd.read_csv(csv_path)
    groups = df['pid']  # Assuming 'pid' is the column for patient IDs
    y = df['label']     # Assuming 'label' is the target variable

    # folder = GroupKFold(n_splits=5)
    folder = KFold(n_splits=5, shuffle=True, random_state=random_seed)
    folds = []

    # for train_idx, val_idx in folder.split(df, y, groups):
    for train_idx, val_idx in folder.split(df, y, groups):
        train_df = df.iloc[train_idx].reset_index(drop=True)
        val_df = df.iloc[val_idx].reset_index(drop=True)
        fold = {'train': train_df, 'val': val_df}
        folds.append(fold)
    return folds

folds = k_fold_split(csv_path = '/mnt/hdd/octc/experiment/dataset-train.csv')
for fold in folds:
    # train과 valid에서 겹치는 pid 개수 확인
    print(len(set(fold['train']['pid'].unique()) & set(fold['val']['pid'].unique())))